# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
# with reset_index()
purchase_per_customer = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
purchase_per_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [5]:
# with as_index argument in .groupby method
purchase_per_customer2 = data.groupby(['CustomerID','ProductName'],as_index=False)['Quantity'].sum()
purchase_per_customer2.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
# with fill_value=0 argument of pd.pivot_table method --> 0 in int format
# with Total (All) column (margins=True)
product_by_customer = purchase_per_customer.pivot_table(index='ProductName',columns="CustomerID",
                                     values="Quantity",aggfunc=np.sum,fill_value=0, margins=True)
display(product_by_customer.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,All
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,0,1918
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,0,0,0,0,0,0,0,0,2228
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,25,0,1880
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,25,25,0,25,0,1738
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,25,0,0,0,0,0,0,0,0,2035


In [7]:
# with fillna(0) method --> 0 in float format
# without Total (All) column (margins=True)
product_by_customer2 = data.pivot_table(index='ProductName',columns="CustomerID",
                                     values="Quantity",aggfunc=np.sum).fillna(0)
product_by_customer2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
transposed = product_by_customer.T
transposed.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow,All
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,63
200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,77
264,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,65
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,67
412,0,0,0,0,1,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,73


In [9]:
transposed2 = product_by_customer2.T
transposed2.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
squareform_distance_matrix = squareform(pdist(transposed, 'euclidean'))
squareform_distance_matrix

array([[0.00000000e+00, 1.83847763e+01, 1.06770783e+01, ...,
        2.00132206e+03, 1.77691868e+03, 8.86422779e+05],
       [1.83847763e+01, 0.00000000e+00, 1.67928556e+01, ...,
        1.98742748e+03, 1.76302808e+03, 8.86408764e+05],
       [1.06770783e+01, 1.67928556e+01, 0.00000000e+00, ...,
        1.99924886e+03, 1.77493549e+03, 8.86420779e+05],
       ...,
       [2.00132206e+03, 1.98742748e+03, 1.99924886e+03, ...,
        0.00000000e+00, 3.77491722e+02, 8.84433559e+05],
       [1.77691868e+03, 1.76302808e+03, 1.77493549e+03, ...,
        3.77491722e+02, 0.00000000e+00, 8.84658836e+05],
       [8.86422779e+05, 8.86408764e+05, 8.86420779e+05, ...,
        8.84433559e+05, 8.84658836e+05, 0.00000000e+00]])

In [11]:
squareform_distance_matrix2 = squareform(pdist(transposed2, 'euclidean'))
squareform_distance_matrix2

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [21]:
distances = pd.DataFrame(1/(1 +squareform_distance_matrix),
            index=product_by_customer.columns, columns=product_by_customer.columns)
distances.sort_values(by='All',ascending=False).head() # ordered by total

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,All
CustomerID,,,,,,,,,,,,,,,,,,,,,
All,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.000000
91777,0.000479,0.000482,0.000479,0.000480,0.000481,0.000480,0.000482,0.000478,0.000480,0.000480,...,0.001701,0.002010,0.001300,0.002194,0.001874,0.002072,0.002020,0.003148,0.002282,0.000001
91443,0.000485,0.000488,0.000485,0.000486,0.000487,0.000485,0.000488,0.000484,0.000485,0.000486,...,0.001742,0.002141,0.001345,0.002292,0.001997,0.002130,0.002112,0.003137,0.002336,0.000001
97324,0.000487,0.000490,0.000488,0.000488,0.000489,0.000488,0.000490,0.000487,0.000488,0.000488,...,0.001761,0.002184,0.001357,0.002259,0.001986,0.002127,0.002110,0.003213,0.002376,0.000001
92255,0.000496,0.000499,0.000496,0.000497,0.000498,0.000496,0.000499,0.000495,0.000496,0.000497,...,0.001894,0.002258,0.001422,0.002411,0.002096,0.002287,0.002279,0.003199,0.002548,0.000001


In [20]:
distances2 = pd.DataFrame(1/(1 +squareform_distance_matrix2),
            index=product_by_customer2.columns, columns=product_by_customer2.columns)
distances2.sort_values(by=33,ascending=False).head() # ordered by first column

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
3317,0.087047,0.084959,0.084297,0.085638,0.085638,0.086688,0.081210,0.088530,0.085297,0.083651,...,0.004806,0.004685,0.004415,0.005039,0.004309,0.004525,0.004579,0.004360,0.004164,0.004332
3535,0.087047,0.082403,0.086333,0.082403,0.084959,0.085983,0.083651,0.087779,0.087410,0.084959,...,0.004817,0.004677,0.004415,0.005036,0.004317,0.004522,0.004582,0.004364,0.004169,0.004348
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
2503,0.085983,0.080350,0.082709,0.083333,0.080350,0.085638,0.077171,0.085983,0.085638,0.081503,...,0.004814,0.004670,0.004410,0.005020,0.004303,0.004534,0.004581,0.004355,0.004173,0.004324


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [22]:
top33 = distances[33].sort_values(ascending=False)[1:6]
top33

CustomerID
264     0.085638
3909    0.084297
3885    0.082403
3074    0.082403
639     0.081800
Name: 33, dtype: float64

In [30]:
# Top 5 most similar customers for client ID 200
similarities_client33 = distances2.nlargest(6, [33])[1:6]
similarities_client33

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
3317,0.087047,0.084959,0.084297,0.085638,0.085638,0.086688,0.081210,0.088530,0.085297,0.083651,...,0.004806,0.004685,0.004415,0.005039,0.004309,0.004525,0.004579,0.004360,0.004164,0.004332
3535,0.087047,0.082403,0.086333,0.082403,0.084959,0.085983,0.083651,0.087779,0.087410,0.084959,...,0.004817,0.004677,0.004415,0.005036,0.004317,0.004522,0.004582,0.004364,0.004169,0.004348
2503,0.085983,0.080350,0.082709,0.083333,0.080350,0.085638,0.077171,0.085983,0.085638,0.081503,...,0.004814,0.004670,0.004410,0.005020,0.004303,0.004534,0.004581,0.004355,0.004173,0.004324
3305,0.085638,0.077421,0.084297,0.084297,0.080070,0.082709,0.077421,0.086333,0.082100,0.081800,...,0.004814,0.004672,0.004419,0.005029,0.004308,0.004527,0.004576,0.004359,0.004171,0.004336


In [31]:
# Top 5 most similar customers for client ID 200
similarities_client200 = distances2[200].sort_values(ascending=False)[1:]
similarities_client200.head()

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.